In [79]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class myconv_net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv1d(2,2,200)  ## Channels 2 as real and imaginary for inp & out, 100 is filter length being applied , 839-200+1 = 640
    self.conv2 = nn.Conv1d(2,2,100)   ## 640-100+1 = 541
    self.conv3 = nn.Conv1d(2,2,50)   ## 541-50+1 = 492
    self.conv4 = nn.Conv1d(2,2,128)   ## 492-128+1 = 365
    self.conv5 = nn.Conv1d(2,2,64)    ## 365 -64+1 = 302

    self.fc1 = nn.Linear(302,256)
    self.fc2 = nn.Linear(256,128)
  def forward(self, x):
        # x shape: [B, 839, 2]
        x = x.permute(0, 2, 1)  # → [B, 2, 839]

        x = F.relu(self.conv1(x))  # → [B, 2, 640]
        x = F.relu(self.conv2(x))  # → [B, 2, 541]
        x = F.relu(self.conv3(x))  # → [B, 2, 492]
        x = F.relu(self.conv4(x))  # → [B, 2, 365]
        x = F.relu(self.conv5(x))  # → [B, 2, 302]

        # 🔥 Max over channels (dim=1)
        x = torch.max(x, dim=1)[0]  # → [B, 302]

        x = F.relu(self.fc1(x))     # → [B, 256]
        x = self.fc2(x)             # → [B, num_classes]

        return x

model = myconv_net()
print(model)

myconv_net(
  (conv1): Conv1d(2, 2, kernel_size=(200,), stride=(1,))
  (conv2): Conv1d(2, 2, kernel_size=(100,), stride=(1,))
  (conv3): Conv1d(2, 2, kernel_size=(50,), stride=(1,))
  (conv4): Conv1d(2, 2, kernel_size=(128,), stride=(1,))
  (conv5): Conv1d(2, 2, kernel_size=(64,), stride=(1,))
  (fc1): Linear(in_features=302, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
)


In [80]:
import torch.optim as optim
criteria = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [81]:
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/MyDrive/Colab Notebooks')
print("Loading Data")
labels = pd.read_csv('Labels.csv', header=None)
Training = pd.read_csv('Training_Data.csv', header=None)
# Stack real and imaginary components in the desired format
num_samples = Training.shape[0]  # Get the number of samples
num_features = Training.shape[1]  # Get the number of original features
Training = Training.applymap(lambda x: complex(str(x).replace("i","j")))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Loading Data


/tmp/ipython-input-281073872.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  Training = Training.applymap(lambda x: complex(str(x).replace("i","j")))


In [82]:
real = Training.applymap(lambda x: x.real)
imag = Training.applymap(lambda x: x.imag)


/tmp/ipython-input-242914066.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  real = Training.applymap(lambda x: x.real)
/tmp/ipython-input-242914066.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  imag = Training.applymap(lambda x: x.imag)


In [105]:
import numpy as np
X = torch.tensor(np.stack([real.to_numpy(), imag.to_numpy()],axis=-1), dtype=torch.float32)
print(X.shape)
Y = torch.tensor(labels.transpose().to_numpy(), dtype=torch.long)
print(Y.shape)
Y = np.argmax(Y, axis=1)


torch.Size([4096, 839, 2])
torch.Size([4096, 128])


In [106]:
from sklearn.model_selection import train_test_split
## 0.8 train, 0.1 Val, 0.1 Test

X_temp, X_test = train_test_split(X, test_size=0.1, random_state=42)
X_train, X_val = train_test_split(X, test_size=0.111, random_state=42)

Y_temp, Y_test = train_test_split(Y, test_size=0.1, random_state=42)
Y_train, Y_val = train_test_split(Y, test_size=0.111, random_state=42)

In [107]:
from torch.utils.data import DataLoader, Dataset

class MyDataset(Dataset):
  def __init__(self,X,Y):
    self.X = X
    self.Y = Y
  def __len__(self):
    return len(self.X)
  def __getitem__(self,idx):
    return self.X[idx], self.Y[idx]

train_dataset = MyDataset(X_train,Y_train)
val_dataset = MyDataset(X_val,Y_val)
test_dataset = MyDataset(X_test,Y_test)

In [108]:
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=64, shuffle=True)
print(trainloader.dataset.X.__getitem__(0).shape)

torch.Size([839, 2])


In [109]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
def train(model, train_loader, val_loader, criteria, optimizer, num_epochs=1):
    model.train()
    for epoch in range(num_epochs):
      running_loss = 0.0
      correct = 0
      total = 0
      for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criteria(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

      train_loss = running_loss / total
      train_acc = correct / total

      # Validation at end of epoch
      val_acc = evaluate(model, val_loader)

      print(f"Epoch {epoch+1}: "
            f"Train Loss={train_loss:.4f}, "
            f"Train Acc={train_acc:.4f}, "
            f"Val Acc={val_acc:.4f}")

In [110]:
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    model.train()  # Switch back to training mode
    return correct / total

In [111]:
print(X_test.shape)
print(Y_test.shape)

torch.Size([410, 839, 2])
torch.Size([410])


In [113]:
train(model, trainloader, valloader, criteria, optimizer, num_epochs=50)


Epoch 1: Train Loss=0.4189, Train Acc=0.8871, Val Acc=0.9231
Epoch 2: Train Loss=0.2474, Train Acc=0.9374, Val Acc=0.9648
Epoch 3: Train Loss=0.1375, Train Acc=0.9662, Val Acc=0.9692
Epoch 4: Train Loss=0.1687, Train Acc=0.9566, Val Acc=0.9802
Epoch 5: Train Loss=0.0884, Train Acc=0.9791, Val Acc=0.9824
Epoch 6: Train Loss=0.0386, Train Acc=0.9923, Val Acc=0.9868
Epoch 7: Train Loss=0.0759, Train Acc=0.9800, Val Acc=0.9802
Epoch 8: Train Loss=0.0393, Train Acc=0.9901, Val Acc=0.9516
Epoch 9: Train Loss=0.1062, Train Acc=0.9709, Val Acc=0.9780
Epoch 10: Train Loss=0.0307, Train Acc=0.9920, Val Acc=0.9956
Epoch 11: Train Loss=0.0513, Train Acc=0.9876, Val Acc=0.9890
Epoch 12: Train Loss=0.0179, Train Acc=0.9973, Val Acc=0.9978
Epoch 13: Train Loss=0.0572, Train Acc=0.9882, Val Acc=0.9582
Epoch 14: Train Loss=0.0431, Train Acc=0.9887, Val Acc=0.9912
Epoch 15: Train Loss=0.0488, Train Acc=0.9857, Val Acc=0.9648
Epoch 16: Train Loss=0.0372, Train Acc=0.9896, Val Acc=0.9978
Epoch 17: Train L

In [114]:
torch.save(model.state_dict(), "convnet_model.pt")
test_acc = evaluate(model, testloader)
print(f"Test Accuracy: {test_acc:.4f}")

Test Accuracy: 1.0000
